In [ ]:
import numpy as np
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec

In [ ]:
# opening the text file
path = '/content/t8.shakespeare.output.txt'
text = open(path).read().lower()
print('length of the corpus is: :', len(text))

word_text = []
word_count = []

with open(path,'r') as file:

  # reading each line
  for line in file:

    # reading each word
    for word in line.split():

      # storing the words
      if word.isnumeric():
        word_count.append(word)

      else:
        word = word.replace(":","")
        if word not in stopwords:
          word_text.append(word)
        else:
          word_count.pop()

print((word_text))
print(len(word_count))

word_count = [int(x) for x in word_count]

In [ ]:
df = pd.DataFrame()
df['word_text'] = word_text
df['word_count'] = word_count
print(len(df))

df = df.sort_values(by="word_count", ascending=False, kind="mergesort" )
df = df.head(500)
print(len(df))

In [ ]:
wlist = []
word_list = df["word_text"].values.tolist()
wlist.append(word_list)

# Load pre-trained word embeddings model
model = Word2Vec(sentences=wlist, vector_size=1, window=5, min_count=1, workers=4)
model.save("word2vec.model")

embedding_model = Word2Vec.load("word2vec.model")

In [ ]:
word_vectors = [embedding_model.wv[word] for word in word_list]
similarity_scores = []
for i, word_vector in enumerate(word_vectors):
  wv_list = []
  wv_list.append(word_vector)
  other_vectors = word_vectors[:i] + word_vectors[i+1:]
  for other_vector in other_vectors:
    ov_list = []
    ov_list.append(other_vector)
    #print(cosine_similarity(wv_list, ov_list))
    similarity_scores = cosine_similarity(wv_list, ov_list).flatten()
    context_similarity = np.mean([similarity_scores])
  similarity_scores = np.append(similarity_scores, context_similarity)


In [ ]:
top_k = 10

# Get top-k words based on similarity scores
context_indices = np.argsort(similarity_scores)[-top_k:]
context = [word_list[index] for index in context_indices]

# Display the extracted context
print("Extracted context from the given word cloud:")
print(context)